### Configurig Models and Vars

In [ ]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Only need to go up one level since we're in the Python Notebooks directory
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the application code from Day 3 to provide context for test generation
# TODO: potentially change this path depending on project structure
main_code = load_artifact("Artifacts/backend/app/main.py")
crud_code = load_artifact("Artifacts/backend/app/crud.py")
if not main_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-11-05 16:10:16,530 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None


### Generating Tests with Fixture
**Task:** Generate `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

In [ ]:
# Generate the pytest fixture for an isolated test database
db_fixture_prompt = f"""
You are a senior QA Engineer. Generate a pytest configuration file (conftest.py) that is compatible with the StaffAlloc FastAPI backend located in `Artifacts/backend/app`.

FastAPI main.py Code:
{main_code}

CRUD Code:
{crud_code}

Context:
- Tests run from the repository root, so the backend package directory `Artifacts/backend` must be added to `sys.path` in the fixture setup.
- The FastAPI application expects to be imported as the package `app`.

Requirements:
1. Output a `conftest.py` file that configures pytest fixtures for fully isolated database testing.
2. Use an in-memory SQLite database (`sqlite:///:memory:`) with SQLAlchemy.
3. Import the necessary modules: `pytest`, `TestClient` from `fastapi.testclient`, `create_engine`, `sessionmaker`, and `StaticPool` from SQLAlchemy.
4. Add `Artifacts/backend` to `sys.path` so `import app` works when tests run from the repo root.
5. Import `Base` from `app.models`, `create_app` from `app.main`, `settings` from `app.core.config`, and `get_db` from `app.db.session`.
6. Before creating the app instance, set `settings.API_V1_STR = ""` to remove the version prefix from route paths during tests, then call `create_app()` to get the FastAPI app.
7. Create a database engine with `StaticPool` for the in-memory SQLite database.
8. Create a `TestingSessionLocal` sessionmaker bound to the test engine.
9. Define an `override_get_db()` generator that yields a test database session and closes it afterward.
10. Override the app's `get_db` dependency: `app.dependency_overrides[get_db] = override_get_db`.
11. Define a pytest fixture named `client` with `scope="function"` that:
    - Creates all database tables using `Base.metadata.create_all(bind=engine)` before each test.
    - Yields a `TestClient(app)` instance.
    - Drops all tables using `Base.metadata.drop_all(bind=engine)` after each test.
12. Provide additional fixtures for common test data:
    - `user_data`: Returns a dict with valid user test data.
    - `role_data`: Returns a dict with valid role test data.
    - `lcat_data`: Returns a dict with valid LCAT test data.
    - `project_data`: Returns a dict with valid project test data.

Output only valid Python code for `conftest.py` with proper imports and fixtures. This file will be used by all test files.
"""

print("--- Generating Pytest DB Fixture ---")
if main_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "Artifacts/Tests/conftest.py", overwrite=True)
else:
    print("Skipping fixture generation because app context is missing.")

# Create comprehensive happy path tests for all CRUD operations
happy_path_tests_prompt = f"""
You are a senior QA Engineer. Create comprehensive happy path tests for all api operations in a FastAPI application using pytest fixtures.

FastAPI Application Code:
{main_code}


Requirements:
1. Import pytest and TestClient from fastapi.testclient
2. Use the fixtures defined in conftest.py (client, user_data, role_data, lcat_data, project_data)
3. Create test functions for each CRUD operation in the following modules:

Users:
- test_create_user: Create a user and verify all fields
- test_get_user: Create and then retrieve a user
- test_get_users: Create multiple users and list them
- test_update_user: Create, update, and verify changes
- test_delete_user: Create and then delete a user

Roles:
- test_create_role: Create a role and verify fields
- test_get_role: Create and retrieve a role
- test_get_roles: Create multiple roles and list them
- test_update_role: Create, update, and verify changes
- test_delete_role: Create and delete a role

LCATs:
- test_create_lcat: Create an LCAT and verify fields
- test_get_lcat: Create and retrieve an LCAT
- test_get_lcats: Create multiple LCATs and list them
- test_update_lcat: Create, update, and verify changes
- test_delete_lcat: Create and delete an LCAT

Projects:
- test_create_project: Create a project and verify fields
- test_get_project: Create and retrieve a project
- test_get_projects: Create multiple projects and list them
- test_update_project: Create, update, and verify changes
- test_delete_project: Create and delete a project
- test_get_projects_for_user: Create project assignments and verify user's projects
- test_get_projects_managed_by_user: Create projects with manager and verify

ProjectAssignments:
- test_create_project_assignment: Create assignment and verify fields
- test_get_project_assignment: Create and retrieve an assignment
- test_get_project_assignments: Create multiple assignments and list them
- test_update_project_assignment: Create, update, and verify changes
- test_delete_project_assignment: Create and delete an assignment
- test_get_assignments_for_project: Create assignments and verify project's assignments
- test_get_assignments_for_user: Create assignments and verify user's assignments

Allocations:
- test_create_allocation: Create allocation and verify fields
- test_get_allocation: Create and retrieve an allocation
- test_get_allocations: Create multiple allocations and list them
- test_update_allocation: Create, update, and verify changes
- test_delete_allocation: Create and delete an allocation
- test_get_allocations_for_assignment: Create allocations and verify assignment's allocations
- test_get_user_allocation_summary: Create allocations and verify user's summary

For each test:
1. Use the appropriate fixture for test data
2. Create any required parent/related entities first
3. Make the API call using the client fixture
4. Assert the correct status code (usually 200 for success)
5. Verify the response data matches the input data
6. For list endpoints, verify pagination works
7. For delete operations, verify the item cannot be retrieved afterward

Output Format:
- Group tests by entity using comments (e.g., # User Tests, # Role Tests, etc.)
- Include proper type hints for the client fixture
- Use descriptive test names that indicate the operation being tested
- Add brief docstrings explaining the test's purpose
- Handle any required relationships between entities

Output only valid Python test functions that use the fixtures from conftest.py.
"""

print("\n--- Generating Happy Path Tests ---")
if main_code:
    happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "Artifacts/tests/test_crud_happy_path.py", overwrite=True)
else:
    print("Skipping test generation because app context is missing.")

--- Generating Pytest DB Fixture ---
# conftest.py

"""
Pytest configuration file for the FastAPI application.

This file sets up fixtures for database testing, ensuring each test runs in
complete isolation with a fresh in-memory database.

Key components:
- In-memory SQLite database engine (`sqlite:///:memory:`).
- `StaticPool` to maintain a single connection for the in-memory DB's lifetime.
- A dependency override for FastAPI's `get_db` to use the test database session.
- A function-scoped `client` fixture that creates and tears down the database
  schema for each test, providing a `TestClient` instance.
- Reusable data fixtures for common models to keep tests DRY.
"""

import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import StaticPool

# Assume your main application, SQLAlchemy Base, and get_db dependency
# are defined in `main.py`. Adjust the import path if your project
# struct

### Generating Edge Cases

In [ ]:
# TODO: CHANGE THIS PROMPT and ADD GENERATED EDGE CASES TO TESTING FILE
edge_case_tests_prompt = f"""
You are a senior QA Engineer. Generate pytest test functions for edge cases and error scenarios in a FastAPI application.

FastAPI Application Code:
{app_code}

Requirements:
1. Use FastAPI's TestClient from fastapi.testclient import TestClient
2. Import the FastAPI app instance (same setup as happy path tests)
3. Write a test function for POST /users/ that:
   - First creates a user with a valid email
   - Then attempts to create another user with the same email
   - Asserts status code is 400 (Bad Request)
   - Verifies the error message indicates duplicate email
4. Write a test function for GET /users/{{user_id}} that:
   - Requests a user ID that does not exist (e.g., 99999 or a high number)
   - Asserts status code is 404 (Not Found)
   - Verifies an appropriate error message

Output only valid Python pytest functions with proper imports. Use descriptive function names like test_create_user_duplicate_email and test_get_user_not_found. Ensure tests use the same database setup pattern as happy path tests.
"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")